<a href="https://colab.research.google.com/github/lilgabi05/teste-anhanguera/blob/main/MEDCALL_VIDAS%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
from collections import deque
from datetime import datetime
import uuid

# ==============================================================================
# 1. ARMAZENAMENTO DE DADOS (Dicionários Centrais)
# ==============================================================================

# Dicionários onde a chave é o ID (ou chave simplificada) e o valor é o registro completo.
pacientes_db = {}   # {id: {'nome': str, 'idade': int, 'telefone': str, 'historico_atendimentos': []}}
medicos_db = {}     # {crm: {'nome': str, 'especialidade': str}}
exames_db = {}      # {id: {'nome': str, 'preco': float}}

# Lista complexa para agendamentos (não usa chave, é percorrida em busca)
agendamentos_db = []

# ==============================================================================
# MOCK DATA (Dados Iniciais para Teste)
# ==============================================================================

def inicializar_dados():
    """Popula os dicionários com alguns dados iniciais para teste."""
    global medicos_db, exames_db, pacientes_db

    # Médicos
    medicos_db = {
        'CRM123': {'nome': 'Dr. Almeida', 'especialidade': 'Clínico Geral'},
        'CRM456': {'nome': 'Dra. Luiza', 'especialidade': 'Cardiologia'}
    }

    # Exames
    exames_db = {
        'hemograma': {'nome': 'Hemograma Completo', 'preco': 50.0},
        'eletro': {'nome': 'Eletrocardiograma', 'preco': 80.0}
    }

    # Pacientes
    pacientes_db = {
        'ana_silva': {
            'nome': 'Ana Silva',
            'idade': 35,
            'telefone': '9999-1111',
            'historico_atendimentos': [
                {'data': '10/11/2025 10:00', 'medico_crm': 'CRM123', 'diagnostico': 'Gripe sazonal', 'evolucao': 'Prescrito Paracetamol.'}
            ]
        }
    }

# ==============================================================================
# 2. FUNÇÕES DE CADASTRO (Paciente, Médico, Exame)
# ==============================================================================

def _obter_input_seguro(prompt, tipo=str, limite=None):
    """Função auxiliar para tratar entradas com tipagem e validação básica."""
    while True:
        try:
            valor = input(prompt).strip()
            if not valor and tipo != str:
                return None
            if tipo == str and not valor:
                return None

            if tipo == int:
                valor = int(valor)
                if limite and not (0 <= valor <= limite):
                    print(f"ERRO: Valor fora do limite (0-{limite}).")
                    continue
            elif tipo == float:
                valor = float(valor)
                if valor < 0:
                     print("ERRO: O valor não pode ser negativo.")
                     continue

            return valor
        except ValueError:
            print(f"ERRO: Entrada inválida. Esperado um(a) {tipo.__name__}.")
        except Exception:
            return None

def cadastrar_paciente():
    """Permite o cadastro de pacientes."""
    print("\n--- [ Cadastro de Paciente ] ---")
    nome = input("Nome completo: ").strip().title()
    if not nome: return print("ERRO: Nome obrigatório.")

    idade = _obter_input_seguro("Idade: ", tipo=int, limite=120)
    if idade is None: return

    telefone = input("Telefone: ").strip()

    # Chave simples (Nome_Sobrenome) para facilitar a busca
    paciente_id = nome.replace(" ", "_").lower()

    pacientes_db[paciente_id] = {
        'nome': nome,
        'idade': idade,
        'telefone': telefone,
        'historico_atendimentos': []
    }
    print(f"SUCESSO: Paciente '{nome}' cadastrado com ID: {paciente_id}")

def cadastrar_medico():
    """Permite o cadastro de médicos."""
    print("\n--- [ Cadastro de Médico ] ---")
    crm = input("CRM (Identificador): ").strip().upper()
    if not crm: return print("ERRO: CRM obrigatório.")
    if crm in medicos_db: return print("ERRO: CRM já cadastrado.")

    nome = input("Nome do Médico: ").strip().title()
    especialidade = input("Especialidade: ").strip().title()

    medicos_db[crm] = {
        'nome': nome,
        'especialidade': especialidade
    }
    print(f"SUCESSO: Médico(a) {nome} ({especialidade}) cadastrado(a).")

def cadastrar_exame():
    """Permite o cadastro de exames."""
    print("\n--- [ Cadastro de Exame ] ---")
    nome = input("Nome do Exame: ").strip().title()
    if not nome: return print("ERRO: Nome obrigatório.")

    preco = _obter_input_seguro("Preço do Exame (R$): ", tipo=float)
    if preco is None: return

    exame_id = nome.replace(" ", "_").lower()
    if exame_id in exames_db: return print("ERRO: Exame já cadastrado.")

    exames_db[exame_id] = {
        'nome': nome,
        'preco': preco
    }
    print(f"SUCESSO: Exame '{nome}' (R$ {preco:.2f}) cadastrado.")

# ==============================================================================
# 3. AGENDAMENTO E ATENDIMENTO
# ==============================================================================

def agendar_servico():
    """Permite agendar consulta ou exame, com checagem básica de disponibilidade."""
    print("\n--- [ Agendamento de Serviço ] ---")

    paciente_nome = input("Nome do Paciente para agendar: ").strip().title()
    paciente_id = paciente_nome.replace(" ", "_").lower()
    if paciente_id not in pacientes_db:
        print(f"ERRO: Paciente '{paciente_nome}' não encontrado.")
        return

    tipo = input("Tipo (Consulta / Exame): ").strip().title()
    if tipo not in ["Consulta", "Exame"]:
        print("ERRO: Tipo deve ser 'Consulta' ou 'Exame'.")
        return

    data_str = input("Data (DD/MM/AAAA): ").strip()
    hora_str = input("Hora (HH:MM): ").strip()

    try:
        data_hora = datetime.strptime(f"{data_str} {hora_str}", "%d/%m/%Y %H:%M")
        if data_hora < datetime.now():
            print("ERRO: Não é possível agendar no passado.")
            return
    except ValueError:
        print("ERRO: Formato de data/hora inválido.")
        return

    # Lógica SIMPLIFICADA de Controle de Horários: checa se já há algo agendado no mesmo horário
    for agendamento in agendamentos_db:
        if agendamento['data_hora'] == data_hora:
            print("ERRO: Já existe um agendamento para este horário.")
            return

    medico_crm = "N/A"
    exame_id = "N/A"

    if tipo == "Consulta":
        medico_nome = input("Nome do Médico: ").strip().title()
        medico_crm = medico_nome.replace(" ", "_").upper()
        if medico_crm not in medicos_db:
            print(f"ALERTA: Médico '{medico_nome}' não encontrado. O agendamento continua, mas verifique o cadastro.")

    elif tipo == "Exame":
        exame_nome = input("Nome do Exame: ").strip().title()
        exame_id = exame_nome.replace(" ", "_").lower()
        if exame_id not in exames_db:
            print(f"ALERTA: Exame '{exame_nome}' não encontrado. O agendamento continua, mas verifique o cadastro.")


    novo_agendamento = {
        'paciente_id': paciente_id,
        'medico_crm': medico_crm,
        'exame_id': exame_id,
        'tipo': tipo,
        'data_hora': data_hora,
        'status': 'AGENDADO'
    }
    agendamentos_db.append(novo_agendamento)
    print(f"SUCESSO: {tipo} agendado para {pacientes_db[paciente_id]['nome']} às {data_hora.strftime('%d/%m/%Y %H:%M')}")

def registrar_atendimento():
    """Registra a evolução de um atendimento e armazena no histórico do paciente."""
    print("\n--- [ Registro de Atendimento ] ---")

    paciente_nome = input("Nome do Paciente para registro: ").strip().title()
    paciente_id = paciente_nome.replace(" ", "_").lower()
    if paciente_id not in pacientes_db:
        print(f"ERRO: Paciente '{paciente_nome}' não encontrado.")
        return

    data_atendimento = datetime.now().strftime("%d/%m/%Y %H:%M")
    medico_crm = input("CRM do Médico que atendeu: ").strip().upper()
    diagnostico = input("Diagnóstico/Motivo: ").strip()
    evolucao = input("Evolução/Procedimento realizado: ").strip()

    if not diagnostico or not evolucao:
        print("ERRO: Diagnóstico e Evolução são obrigatórios.")
        return

    novo_registro = {
        'data': data_atendimento,
        'medico_crm': medico_crm,
        'diagnostico': diagnostico,
        'evolucao': evolucao
    }

    # Armazenamento no histórico do paciente
    pacientes_db[paciente_id]['historico_atendimentos'].append(novo_registro)

    print(f"SUCESSO: Atendimento de {paciente_nome} registrado na data {data_atendimento}.")

def visualizar_historico_paciente():
    """Exibe o histórico de atendimentos de um paciente específico."""
    print("\n--- [ Histórico do Paciente ] ---")

    paciente_nome = input("Nome do Paciente para visualizar histórico: ").strip().title()
    paciente_id = paciente_nome.replace(" ", "_").lower()

    if paciente_id not in pacientes_db:
        print(f"ERRO: Paciente '{paciente_nome}' não encontrado.")
        return

    historico = pacientes_db[paciente_id]['historico_atendimentos']

    if not historico:
        print(f"INFO: O paciente {paciente_nome} não possui histórico de atendimentos.")
        return

    print(f"\nHistórico de Atendimentos de {pacientes_db[paciente_id]['nome']}:")
    for i, registro in enumerate(historico, 1):
        print("-" * 40)
        print(f"{i}. Data: {registro['data']}")
        print(f"   Médico (CRM): {registro['medico_crm']}")
        print(f"   Diagnóstico: {registro['diagnostico']}")
        print(f"   Evolução: {registro['evolucao']}")
    print("-" * 40)

# ==============================================================================
# 4. GERAÇÃO DE RELATÓRIOS
# ==============================================================================

def gerar_relatorio_mensal():
    """Gera um relatório administrativo resumido de um mês específico."""
    print("\n--- [ Geração de Relatório Mensal ] ---")

    mes_str = input("Mês do Relatório (MM/AAAA): ").strip()
    if len(mes_str) != 7 or '/' not in mes_str:
        print("ERRO: Formato inválido. Use MM/AAAA.")
        return

    total_pacientes = len(pacientes_db)

    # Contagem de Atendimentos no mês (Busca no histórico)
    atendimentos_no_mes = 0
    for paciente in pacientes_db.values():
        for registro in paciente['historico_atendimentos']:
            if registro['data'].startswith(mes_str): # Verifica se o registro começa com MM/AAAA
                atendimentos_no_mes += 1

    # Contagem de Agendamentos no mês
    agendamentos_no_mes = 0
    for agendamento in agendamentos_db:
        data_agendamento = agendamento['data_hora'].strftime("%m/%Y")
        if data_agendamento == mes_str:
            agendamentos_no_mes += 1

    # Média de Idade
    if pacientes_db:
        idades = [p['idade'] for p in pacientes_db.values()]
        idade_media = sum(idades) / len(idades)
    else:
        idade_media = 0

    print("\n" + "="*50)
    print(f"RELATÓRIO ADMINISTRATIVO - MÊS: {mes_str}")
    print("="*50)
    print(f"Pacientes Totais Cadastrados: {total_pacientes}")
    print(f"Média de Idade da Base: {idade_media:.2f} anos")
    print("-" * 50)
    print(f"Atendimentos/Registros no Mês: {atendimentos_no_mes}")
    print(f"Agendamentos Programados no Mês: {agendamentos_no_mes}")
    print(f"Médicos Cadastrados: {len(medicos_db)}")
    print(f"Exames Cadastrados: {len(exames_db)}")
    print("="*50)
    print("RELATÓRIO CONCLUÍDO.")

def listar_agendamentos():
    """Exibe todos os agendamentos futuros."""
    print("\n--- [ Agenda de Consultas/Exames ] ---")
    if not agendamentos_db:
        print("INFO: Nenhuma consulta ou exame agendado.")
        return

    # Ordena por data e hora
    agenda_ordenada = sorted(agendamentos_db, key=lambda x: x['data_hora'])

    print(f"{'Data/Hora':<16} | {'Status':<10} | {'Tipo':<8} | {'Paciente':<15} | {'Detalhes'}")
    print("-" * 75)

    for a in agenda_ordenada:
        p_nome = pacientes_db.get(a['paciente_id'], {'nome': 'Desconhecido'})['nome']
        m_nome = medicos_db.get(a['medico_crm'], {'nome': 'N/A'})['nome']
        exame_nome = exames_db.get(a['exame_id'], {'nome': 'N/A'})['nome']

        info_servico = f"Médico: {m_nome}" if a['tipo'] == 'Consulta' else f"Exame: {exame_nome}"

        print(f"{a['data_hora'].strftime('%d/%m %H:%M'):<16} | {a['status']:<10} | {a['tipo']:<8} | {p_nome:<15} | {info_servico}")
    print("-" * 75)

# ==============================================================================
# 5. PONTO DE ENTRADA
# ==============================================================================

def exibir_menu():
    """Exibe o menu de opções."""
    print("\n" + "="*50)
    print("           SISTEMA MEDCALL VIDA+ - V2.0")
    print("="*50)
    print("--- CADASTRO ---")
    print("1. Cadastrar Paciente")
    print("2. Cadastrar Médico")
    print("3. Cadastrar Exame")
    print("--- OPERAÇÕES ---")
    print("4. Agendar Consulta/Exame")
    print("5. Registrar Atendimento/Evolução")
    print("6. Visualizar Histórico de Paciente")
    print("--- ADMIN/RELATÓRIOS ---")
    print("7. Gerar Relatório Mensal")
    print("8. Listar Agendamentos")
    print("9. Sair do programa")
    print("-" * 50)

def main():
    """Ponto de entrada principal do programa."""
    inicializar_dados() # Carrega os dados iniciais para começar

    while True:
        try:
            exibir_menu()
            escolha = input("Escolha uma opção (1-9): ").strip()

            if escolha == '1': cadastrar_paciente()
            elif escolha == '2': cadastrar_medico()
            elif escolha == '3': cadastrar_exame()
            elif escolha == '4': agendar_servico()
            elif escolha == '5': registrar_atendimento()
            elif escolha == '6': visualizar_historico_paciente()
            elif escolha == '7': gerar_relatorio_mensal()
            elif escolha == '8': listar_agendamentos()
            elif escolha == '9':
                print("\nSistema MEDCALL VIDA+ encerrado.")
                sys.exit(0)
            else:
                print("\nAVISO: Opção inválida. Digite um número de 1 a 9.")

        except KeyboardInterrupt:
            print("\n\nPrograma interrompido. Saindo...")
            sys.exit(0)
        except Exception as e:
            # Em um sistema real, este erro seria mais detalhado
            print(f"\nERRO INESPERADO: {e}. Por favor, verifique a entrada.")

if __name__ == "__main__":
    main()



           SISTEMA MEDCALL VIDA+ - V2.0
--- CADASTRO ---
1. Cadastrar Paciente
2. Cadastrar Médico
3. Cadastrar Exame
--- OPERAÇÕES ---
4. Agendar Consulta/Exame
5. Registrar Atendimento/Evolução
6. Visualizar Histórico de Paciente
--- ADMIN/RELATÓRIOS ---
7. Gerar Relatório Mensal
8. Listar Agendamentos
9. Sair do programa
--------------------------------------------------
